In [3]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

sqlalchemy.__version__

'1.4.32'

In [4]:
connect_str = 'sqlite:////Users/halimaelbadoui/Documents/Ironhack/dataptmad0522_lessons/m1/datasets/publications.db'

In [5]:
engine = create_engine(connect_str)
type(engine)

sqlalchemy.engine.base.Engine

## Challenge 1 - Most Profiting Authors

In [6]:
inspector = inspect(engine)
inspector.get_table_names()

['authors',
 'discounts',
 'employee',
 'jobs',
 'pub_info',
 'publishers',
 'roysched',
 'sales',
 'stores',
 'titleauthor',
 'titles']

In [7]:
sales = pd.read_sql_query(
    "SELECT * FROM sales", engine)
sales
#imprimo la tabla de sales para saber qué columnas tengo (he impreso también titles, titleauthor cambiando la tabla)

,stor_id,ord_num,ord_date,qty,payterms,title_id
0,6380,6871,1994-09-14 00:00:00,5,Net 60,BU1032
1,6380,722a,1994-09-13 00:00:00,3,Net 60,PS2091
2,7066,A2976,1993-05-24 00:00:00,50,Net 30,PC8888
3,7066,QA7442.3,1994-09-13 00:00:00,75,ON invoice,PS2091
4,7067,D4482,1994-09-14 00:00:00,10,Net 60,PS2091
5,7067,P2121,1992-06-15 00:00:00,40,Net 30,TC3218
6,7067,P2121,1992-06-15 00:00:00,20,Net 30,TC4203
7,7067,P2121,1992-06-15 00:00:00,20,Net 30,TC7777
8,7131,N914008,1994-09-14 00:00:00,20,Net 30,PS2091
9,7131,N914014,1994-09-14 00:00:00,25,Net 30,MC3021


In [8]:
query_1 = '''
SELECT 
s.title_id AS 'TITLE ID',
ta.au_id AS 'AUTHOR ID',
ti.advance * ta.royaltyper / 100 AS 'ADVANCE',
ti.price * s.qty * ti.royalty / 100 * ta.royaltyper / 100 AS 'ROYALTY'

FROM Sales s
LEFT JOIN titles ti ON ti.title_id = s.title_id
LEFT JOIN titleauthor ta ON ta.title_id = s.title_id
'''
#la primera query cruzo la tabla sales con la title y titleauthor y aplico las fórmulas para sacar el advance y el royalty

In [9]:
step_1 = pd.read_sql_query(query_1, engine)
step_1

,TITLE ID,AUTHOR ID,ADVANCE,ROYALTY
0,BU1032,213-46-8915,2000,3.998
1,BU1032,409-56-7008,3000,5.997
2,PS2091,899-46-2035,1137,1.971
3,PS2091,998-72-3567,1137,1.971
4,PC8888,427-17-2319,4000,50.000
5,PC8888,846-92-7186,4000,50.000
6,PS2091,899-46-2035,1137,49.275
7,PS2091,998-72-3567,1137,49.275
8,PS2091,899-46-2035,1137,6.570
9,PS2091,998-72-3567,1137,6.570


In [10]:
query_2 = '''
SELECT  
    r.TITLE_ID, 
    r.AUTHOR_ID,
    SUM(r.ROYALTY) AS 'AGG_ROYALTY',
    SUM (r.ADVANCE) AS 'AGG_ADVANCE'
    FROM( SELECT 
        s.title_id AS 'TITLE_ID',
        ta.au_id AS 'AUTHOR_ID',
        ti.advance * ta.royaltyper / 100 AS 'ADVANCE',
        ti.price * s.qty * ti.royalty / 100 * ta.royaltyper / 100 AS 'ROYALTY'

        FROM Sales s
        LEFT JOIN titles ti ON ti.title_id = s.title_id
        LEFT JOIN titleauthor ta ON ta.title_id = s.title_id) r
GROUP BY r.AUTHOR_ID, r.TITLE_ID
''' 
#aquí aplico la query_1 como la subquery de la query_2 donde añado la suma del AVANCE y del ROYALTY y lo agrupo
#por el author id y por el title id

In [11]:
step_2 = pd.read_sql_query(query_2, engine)
step_2

,TITLE_ID,AUTHOR_ID,AGG_ROYALTY,AGG_ADVANCE
0,PS3333,172-32-1176,29.985,2000
1,BU1032,213-46-8915,11.994,4000
2,BU2075,213-46-8915,25.116,10125
3,PC1035,238-95-7766,110.160,7000
4,BU1111,267-41-2394,11.950,2000
5,TC7777,267-41-2394,8.994,2400
6,BU7832,274-80-9391,29.985,5000
7,BU1032,409-56-7008,17.991,6000
8,PC8888,427-17-2319,50.000,4000
9,TC7777,472-27-2349,8.994,2400


In [12]:
query_3 = '''
SELECT
y.AUTHOR_ID,
y.AGG_ROYALTY + y.AGG_ADVANCE AS PROFIT
 
FROM (
    SELECT 
    r.TITLE_ID, 
    r.AUTHOR_ID,
    SUM(r.ROYALTY) AS 'AGG_ROYALTY',
    SUM (r.ADVANCE) AS 'AGG_ADVANCE'
    FROM( SELECT 
        s.title_id AS 'TITLE_ID',
        ta.au_id AS 'AUTHOR_ID',
        ti.advance * ta.royaltyper / 100 AS 'ADVANCE',
        ti.price * s.qty * ti.royalty / 100 * ta.royaltyper / 100 AS 'ROYALTY'

        FROM Sales s
        LEFT JOIN titles ti ON ti.title_id = s.title_id
        LEFT JOIN titleauthor ta ON ta.title_id = s.title_id) r
    GROUP BY r.AUTHOR_ID, r.TITLE_ID) y
''' 

In [13]:
step_3 = pd.read_sql_query(query_3, engine)
step_3

,AUTHOR_ID,PROFIT
0,172-32-1176,2029.985
1,213-46-8915,4011.994
2,213-46-8915,10150.116
3,238-95-7766,7110.160
4,267-41-2394,2011.950
5,267-41-2394,2408.994
6,274-80-9391,5029.985
7,409-56-7008,6017.991
8,427-17-2319,4050.000
9,472-27-2349,2408.994


In [14]:
query_4 = '''
SELECT
x.AUTHOR_ID AS 'AUTHOR_ID',
SUM(x.PROFIT) AS 'TOTAL_PROFIT'
FROM(
    SELECT
    y.AUTHOR_ID,
    y.AGG_ROYALTY + y.AGG_ADVANCE AS PROFIT

    FROM (
        SELECT 
        r.TITLE_ID, 
        r.AUTHOR_ID,
        SUM(r.ROYALTY) AS 'AGG_ROYALTY',
        SUM (r.ADVANCE) AS 'AGG_ADVANCE'
        FROM( SELECT 
            s.title_id AS 'TITLE_ID',
            ta.au_id AS 'AUTHOR_ID',
            ti.advance * ta.royaltyper / 100 AS 'ADVANCE',
            ti.price * s.qty * ti.royalty / 100 * ta.royaltyper / 100 AS 'ROYALTY'

            FROM Sales s
            LEFT JOIN titles ti ON ti.title_id = s.title_id
            LEFT JOIN titleauthor ta ON ta.title_id = s.title_id) r
        GROUP BY r.AUTHOR_ID, r.TITLE_ID) y
)x 
GROUP BY x.AUTHOR_ID
ORDER BY TOTAL_PROFIT DESC
LIMIT 3
''' 

In [15]:
top_profit_authors = pd.read_sql_query(query_4, engine)
top_profit_authors

,AUTHOR_ID,TOTAL_PROFIT
0,722-51-5454,22521.528
1,213-46-8915,14162.110
2,899-46-2035,12126.132
